In [1]:
import math
import numpy as np
from scipy.optimize import newton
from scipy.special import fresnel


# Conversion functions
def db_to_linear(G0_dB):
    return 10 ** (G0_dB / 10)

def linear_to_db(G_linear):
    return 10 * math.log10(G_linear)

#13-56
def horn_equation(chi, lam, a, b, G0):
    lhs = (math.sqrt(2*chi) - b/lam)**2 * (2*chi - 1) 
    rhs = ( (G0/(2*math.pi)) * math.sqrt(3/(2*math.pi)) * (1/math.sqrt(chi)) - (a/lam) )**2 * ( (G0**2)/(6*(math.pi)**3) * (1/chi) - 1)
    result = lhs-rhs
    return result

# 13-57
def initial_chi(G0_linear):
    return G0_linear/(2*math.pi * math.sqrt(2 * math.pi))

# 13-56a
def rho_e(chi, lam):
    return chi * lam

# 13-56b
def rho_h(chi, G0, lam):
    return (G0**2 / (8 * math.pi**3)) * (1 / chi) * lam

# 13-58a
def a1(chi, G0, lam):
    return (G0 / (2 * math.pi)) * math.sqrt(3 / (2 * math.pi * chi)) * lam

# 13-58b
def b1(chi, lam):
    return math.sqrt(2 * chi * lam**2)

#13-49a
def p_e(b, b1, rho_e):
    return (b1 - b) * ( (rho_e/b1)**2 - 1/4) ** (1/2)
    
#13-49b
def p_h(a, a1, rho_h):
    return (a1 - a) * ( (rho_h/a1)**2 - 1/4) ** (1/2)

# pythagoras in geometry
def rho_1(rho_e, b1):
    return math.sqrt(rho_e**2 - (b1/2)**2)

def rho_2(rho_h, a1):
    return math.sqrt(rho_h**2 - (a1/2)**2)

# definition cos in geometry
def psi_e(rho_1, rho_e):
    return math.degrees(math.acos(rho_1/rho_e))

def psi_h(rho_2, rho_h):
    return math.degrees(math.acos(rho_2/rho_h))


# Phase factors
def s_factor(b1_val, lam, rho_1_val):
    return (b1_val**2) / (8 * lam * rho_1_val)

def t_factor(a1_val, lam, rho_2_val):
    return (a1_val**2) / (8 * lam * rho_2_val)

# Fresnel integral numerical integration parameters
Nxi = 4001
xi_points = np.linspace(0.0, 1.0, Nxi)


# Phase losses
def L_h(t):
    taper = np.cos(np.pi * xi_points / 2.0)
    phase_h = np.exp(1j * 2.0 * np.pi * t * xi_points**2)
    integrand_h = taper * phase_h
    int_h = np.trapz(integrand_h, xi_points)
    numerator = 2.0 * int_h
    A_h = numerator * (np.pi / 4.0)
    Lh = np.abs(A_h)**2
    return -linear_to_db(Lh) # convert loss to dB

def L_e(s):
    phase_e = np.exp(1j * 2.0 * np.pi * s * xi_points**2)
    A_e = np.trapz(phase_e, xi_points)
    Le = np.abs(A_e)**2
    return -linear_to_db(Le) # convert loss to dB


# Directivity formula approximation
def D_p(a1, b1, lam, t, s):
    return 10*(1.008 + math.log10( (a1 * b1) / (lam**2) ) ) - (L_h(t) + L_e(s))

### Static case

In [2]:
# Default Parameters
a = 0.14 #m (waveguide)
b = 0.10 #m (waveguide)
lam = 0.211061 #m (wavelength)
G0 = 18.46 #dB (required)

# Book
#lam = 0.027273 #m
#a = 0.02286 #m
#b = 0.01016 #m
#G0 = 22.6 #dB (required)

G0_linear = db_to_linear(G0)
chi_trial = initial_chi(G0_linear)


def horn_optimize(chi):
    return horn_equation(chi, lam, a, b, G0_linear)

optimal_chi = newton(horn_optimize, x0=chi_trial, maxiter=200 )


print("------ CHI STATUS -------")
print("Trial chi: ", chi_trial)
print("Optimal chi:", optimal_chi)
print("Function value at optimal chi:", horn_equation(optimal_chi, lam, a, b, G0_linear))
print("")
print("------ Lengths ------")

rho_e_val = rho_e(optimal_chi, lam)
rho_h_val = rho_h(optimal_chi, G0_linear, lam)
a1_val = a1(optimal_chi, G0_linear, lam)
b1_val = b1(optimal_chi, lam)
rho_1_val = rho_1(rho_e_val, b1_val)
rho_2_val = rho_2(rho_h_val, a1_val)
p_e_val = p_e(b, b1_val, rho_e_val)
p_h_val = p_h(a, a1_val, rho_h_val)


print("rho_e:", rho_e_val, "m")
print("rho_h:", rho_h_val, "m")
print("rho_1:", rho_1_val, "m")
print("rho_2:", rho_2_val, "m")
print("a1:", a1_val, "m")
print("b1:", b1_val, "m")
print("p_e:", p_e_val, "m")
print("p_h:", p_h_val, "m")
print("")

print("------ Taper angles ------")
print("Psi_e =",psi_e(rho_1_val, rho_e_val),"°")
print("Psi_h =",psi_h(rho_2_val, rho_h_val),"°")

print("------ Directivity ------")
t = t_factor(a1_val, lam, rho_2_val)
s = s_factor(b1_val, lam, rho_1_val)
L_h_val = L_h(t)
L_e_val = L_e(s)

print("t-phase error (H-plane): ", t)
print("s-phase error (E-plane): ", s)

print("Phase loss in H-plane (dB): ", (L_h_val))
print("Phase loss in E-plane (dB): ", (L_e_val))
print("Total directivity (dBi): ", D_p(a1_val, b1_val, lam, t, s))


------ CHI STATUS -------
Trial chi:  4.453794734202203
Optimal chi: 4.342345138381712
Function value at optimal chi: -3.979039320256561e-13

------ Lengths ------
rho_e: 0.9164997072519826 m
rho_h: 0.9641487606072241 m
rho_1: 0.862121244974751 m
rho_2: 0.8814545827259367 m
a1: 0.7813338624989731 m
b1: 0.6219925155696179 m
p_e: 0.7235148753811836 m
p_h: 0.7235148753811864 m

------ Taper angles ------
Psi_e = 19.836086705335667 °
Psi_h = 23.903261240511295 °
------ Directivity ------
t-phase error (H-plane):  0.41018084460980636
s-phase error (E-plane):  0.265768797774733
Phase loss in H-plane (dB):  1.198445550681003
Phase loss in E-plane (dB):  1.0955272805162575
Total directivity (dBi):  18.164085395349232


### Iterative case

In [3]:
# Default Parameters
a = 0.14  # m
b = 0.10  # m
lam = 0.211061  # m
G0_start = 18.44  # dB starting value
max_a1 = 1  # m (depth constraint)
max_b1 = 0.9  # m (width constraint)
max_p_e = 0.72 # m

# Step increments
G0_step = 0.0000001   # dB increase per iteration
G0_limit = 60   # dB upper search limit

best_directivity = -1
best_G0 = None
count = 0
G0 = G0_start
while G0 <= G0_limit:
    G0_linear = db_to_linear(G0)
    chi_trial = initial_chi(G0_linear)

    try:
        #print(G0)
        # Solve for chi
        optimal_chi = newton(lambda chi: horn_equation(chi, lam, a, b, G0_linear),
                             x0=chi_trial, maxiter=200)
        
        # Compute horn parameters
        a1_val = a1(optimal_chi, G0_linear, lam)
        b1_val = b1(optimal_chi, lam)
        
        # Check constraints
        if a1_val > max_a1 or b1_val > max_b1:
            break  # stop search if aperture exceeds physical limits

        # Compute phase losses and directivity
        rho_e_val = rho_e(optimal_chi, lam)
        rho_h_val = rho_h(optimal_chi, G0_linear, lam)
        rho_1_val = rho_1(rho_e_val, b1_val)
        rho_2_val = rho_2(rho_h_val, a1_val)
        p_e_val = p_e(b, b1_val, rho_e_val)

        if p_e_val > max_p_e:
            break

        t = t_factor(a1_val, lam, rho_2_val)
        s = s_factor(b1_val, lam, rho_1_val)
        D_total = D_p(a1_val, b1_val, lam, t, s)

        # Track best result
        if D_total > best_directivity:
            best_directivity = D_total
            best_G0 = G0

        count += 1
    except Exception:
        print("Failed chi converging")
        # Skip failed iterations (non-converging chi but cant happen theoretically anyway)
        pass

    G0 += G0_step  # Increase required gain target


print("------ STATUS -------")
print("Took", count, "iterations")
print("Best G0 (dB):", G0)
print("Optimal chi:", optimal_chi)
print("Function value at optimal chi:", horn_equation(optimal_chi, lam, a, b, G0_linear))
print("")
print("------ Lengths ------")

rho_e_val = rho_e(optimal_chi, lam)
rho_h_val = rho_h(optimal_chi, G0_linear, lam)
a1_val = a1(optimal_chi, G0_linear, lam)
b1_val = b1(optimal_chi, lam)
rho_1_val = rho_1(rho_e_val, b1_val)
rho_2_val = rho_2(rho_h_val, a1_val)
p_e_val = p_e(b, b1_val, rho_e_val)
p_h_val = p_h(a, a1_val, rho_h_val)


print("rho_e:", rho_e_val, "m")
print("rho_h:", rho_h_val, "m")
print("rho_1:", rho_1_val, "m")
print("rho_2:", rho_2_val, "m")
print("a1:", a1_val, "m")
print("b1:", b1_val, "m")
print("p_e:", p_e_val, "m")
print("p_h:", p_h_val, "m")
print("")

print("------ Taper angles ------")
print("Psi_e =",psi_e(rho_1_val, rho_e_val),"°")
print("Psi_h =",psi_h(rho_2_val, rho_h_val),"°")
print("")

print("------ Directivity ------")
t = t_factor(a1_val, lam, rho_2_val)
s = s_factor(b1_val, lam, rho_1_val)
L_h_val = L_h(t)
L_e_val = L_e(s)

print("t-phase error (H-plane): ", t)
print("s-phase error (E-plane): ", s)

print("Phase loss in H-plane (dB): ", (L_h_val))
print("Phase loss in E-plane (dB): ", (L_e_val))
print("Total directivity (dBi): ", D_p(a1_val, b1_val, lam, t, s))

------ STATUS -------
Took 21367 iterations
Best G0 (dB): 18.44213670002497
Optimal chi: 4.324171013203024
Function value at optimal chi: -4.334310688136611e-13

------ Lengths ------
rho_e: 0.9126638582176435 m
rho_h: 0.9602689033696206 m
rho_1: 0.8582781861419279 m
rho_2: 0.8775590467416549 m
a1: 0.7797601843145665 m
b1: 0.6206895304083582 m
p_e: 0.7200000061350479 m
p_h: 0.720000006135051 m

------ Taper angles ------
Psi_e = 19.87948112360585 °
Psi_h = 23.954520270052956 °

------ Directivity ------
t-phase error (H-plane):  0.41034371430691674
s-phase error (E-plane):  0.26584150481564334
Phase loss in H-plane (dB):  1.1993635879914009
Phase loss in E-plane (dB):  1.0961371728443605
Total directivity (dBi):  18.144694165735704
